In [1]:
import torchvision 
import torch 
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
model = models.resnet50(pretrained=True)

In [3]:
for isim, param in model.named_parameters():
    
    if 'layer4' not in isim:
        param.requires_grad = False

model.fc = torch.nn.Linear(2048, 2) # resenetin çıkışını multilabel yaptım : red and green (fine tunning)

In [4]:
train_transforms = transforms.Compose([
    #transforms.Grayscale(num_output_channels=3), renkler önemli gray scale yapma
    
    #transforms.RandomResizedCrop(224),
    transforms.Resize((224,224)), # resnet girişi
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])

In [5]:
dataset = datasets.ImageFolder('./dataset', transform=train_transforms)

In [6]:
dataset = datasets.ImageFolder('./dataset', transform = train_transforms)

dataset_boyutu = len(dataset)

egitim_boyutu = int(dataset_boyutu * 0.75)

test_boyutu = dataset_boyutu - egitim_boyutu

egitim_dataset, test_dataset = torch.utils.data.random_split(dataset, [egitim_boyutu, test_boyutu])

print('Dataset boyutu: ', len(dataset))
print('Eğitim boyutu: ', len(egitim_dataset))
print('Test boyutu: ', len(test_dataset))

Dataset boyutu:  2301
Eğitim boyutu:  1725
Test boyutu:  576


In [7]:
egitim_loader = torch.utils.data.DataLoader(egitim_dataset,
                                          batch_size=20,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=20,
                                          shuffle=True)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
total_epoch = 100

for epoch in range(total_epoch):
    epoch_loss = 0
    for X, y in egitim_loader:
        
        optimizer.zero_grad()

        cikti = model(X)
        
        loss = criterion(cikti, y)
        
        epoch_loss += loss.item()
        
        loss.backward()

        optimizer.step()
      
    
    dogru = 0
    toplam = 0
    
    #Test
    with torch.no_grad(): #stop regularization
        for X, y in test_loader:
             
            cikti = model(X)
          
            _, maksimum = torch.max(cikti.data, 1)
          
            toplam += y.size(0)
          
            dogru += (maksimum == y).sum().item()
            
            
    
    accuracy = str(dogru/toplam)
    print("[Epoch]:", epoch, "| Training loss:", epoch_loss, "| Accuracy: ", accuracy)
    
    
    #Save the model
    checkpoint = {'model': model,
                        'state_dict': model.state_dict(),
                        'optimizer' : optimizer.state_dict()}
    

    torch.save(checkpoint, 'models/{}.pth'.format(epoch))
    print("Model kaydedildi.")

E:\PROGRAM FILES\Anaconda\envs\staj_projesi\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


[Epoch]: 0 | Training loss: 8.87125776859466 | Accuracy:  0.9913194444444444
Model kaydedildi.


KeyboardInterrupt: 